# THIS NOTEBOOK IS ON MINI BATCH GD 
BUT ALSO INCLUDES VISULISATION OF GD 

In [1]:
from sklearn.datasets import load_diabetes
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression


In [2]:
X,y = load_diabetes(return_X_y=True)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [4]:
reg = LinearRegression()
reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [5]:
print('this is slope',reg.coef_)
print('this is intercept',reg.intercept_)

this is slope [  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588
  518.06227698  163.41998299  275.31790158  736.1988589    48.67065743]
this is intercept 151.34560453985995


In [7]:
from sklearn.metrics import r2_score
y_pred = reg.predict(X_test)
r2_score(y_test, y_pred)

0.4526027629719197

In [23]:
class MBgdRregressor:
    def __init__(self,batch_size = 10, learning_rate=0.01, epochs= 100):
        self.coef_ = None 
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X_train, y_train):
        #we are here initializing our coef
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        import random
        

        for i in range(self.epochs):
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]), self.batch_size)
                

                y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
                intercept_der = -2 * np.mean(y_train[idx] - y_hat) 
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)
                coef_der = -2 * np.dot((y_train[idx] - y_hat), X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)


        print(self.intercept_, self.coef_)

            
    def predict(self,X_test):
        return np.dot(X_test, self.coef_)+ self.intercept_
        

In [24]:
mbr = MBgdRregressor(batch_size=int(X_train.shape[0]/10), learning_rate=0.01, epochs=50)


In [25]:
mbr.fit(X_train, y_train)

152.34769658060873 [  56.33456266  -86.74106219  359.85748603  261.27955308   -1.08837529
  -38.33911926 -175.67135103  140.34872681  281.7161777   150.6987477 ]


In [26]:
y_pred = mbr.predict(X_test)

In [27]:
r2_score(y_test, y_pred)

0.4457721324305457

# Mini-Batch Gradient Descent (MBGD)

* Uses a small random subset (batch) of data.

* Updates weights per batch.

* Pros: Best of both worlds – faster than BGD, less noisy than SGD.

* Cons: Still depends on batch size tuning.

In [ ]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

# ---------------------
# Mini-Batch Regressor (stores history)
# ---------------------
class MBgdRregressor:
    def __init__(self, batch_size=10, learning_rate=0.01, epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.history = []  # store (coef0, coef1, loss)

    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        self.history = []

        for i in range(self.epochs):
            for j in range(int(X_train.shape[0] / self.batch_size)):
                idx = np.random.choice(X_train.shape[0], self.batch_size, replace=False)
                y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_

                # derivatives
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                coef_der = -2 * np.dot((y_train[idx] - y_hat), X_train[idx])

                # update
                self.intercept_ -= self.lr * intercept_der
                self.coef_ -= self.lr * coef_der

                # compute loss
                full_y_hat = X_train @ self.coef_ + self.intercept_
                loss = np.mean((y_train - full_y_hat) ** 2)

                if len(self.coef_) >= 2:
                    self.history.append((self.coef_[0], self.coef_[1], loss))

    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_


# ---------------------
# Generate synthetic dataset (2D → easy to plot)
# ---------------------
np.random.seed(42)
X = np.random.randn(200, 2)
true_coef = np.array([3.0, -2.0])
y = X @ true_coef + 5 + np.random.randn(200) * 0.5

# ---------------------
# Interactive function
# ---------------------
def run_and_plot(batch_size=20, learning_rate=0.01, epochs=30):
    model = MBgdRregressor(batch_size=batch_size, learning_rate=learning_rate, epochs=epochs)
    model.fit(X, y)
    path = np.array(model.history)

    # Loss surface
    w0 = np.linspace(-1, 6, 80)
    w1 = np.linspace(-5, 3, 80)
    W0, W1 = np.meshgrid(w0, w1)
    Loss = np.zeros_like(W0)

    for i in range(W0.shape[0]):
        for j in range(W0.shape[1]):
            coef = np.array([W0[i, j], W1[i, j]])
            y_hat = X @ coef + 5
            Loss[i, j] = np.mean((y - y_hat) ** 2)

    # Surface
    surface = go.Surface(x=W0, y=W1, z=Loss, colorscale='Viridis', opacity=0.7)

    # Path of descent
    traj = go.Scatter3d(
        x=path[:, 0],
        y=path[:, 1],
        z=path[:, 2],
        mode='lines+markers',
        line=dict(color='red', width=4),
        marker=dict(size=5, color='red')
    )

    fig = go.Figure(data=[surface, traj])
    fig.update_layout(
        scene=dict(
            xaxis_title='Coefficient 0',
            yaxis_title='Coefficient 1',
            zaxis_title='Loss (MSE)'
        ),
        title="Interactive Gradient Descent on Loss Surface"
    )

    fig.show()

# ---------------------
# UI Controls
# ---------------------
batch_slider = widgets.IntSlider(value=20, min=5, max=100, step=5, description='Batch Size:')
lr_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='LR:')
epochs_slider = widgets.IntSlider(value=30, min=10, max=100, step=5, description='Epochs:')

ui = widgets.VBox([batch_slider, lr_slider, epochs_slider])
out = widgets.interactive_output(run_and_plot, {
    'batch_size': batch_slider,
    'learning_rate': lr_slider,
    'epochs': epochs_slider
})

display(ui, out)


Output()

# ALL GD VISULISATIONS

* Batch Gradient Descent (BGD)

Uses all training data to compute the gradient.

Updates weights once per epoch.

Pros: Stable, guaranteed to move toward global minimum (for convex problems).

Cons: Very slow for large datasets.




* Stochastic Gradient Descent (SGD)

Uses one random sample at a time to compute gradient.

Updates weights every sample.

Pros: Fast, escapes local minima, works well for online learning.

Cons: Noisy path, can bounce around instead of smoothly converging.


* Adam (Adaptive Moment Estimation)

Advanced optimizer, extension of SGD.

Uses momentum (history of past gradients) + adaptive learning rate.

Pros: Very fast, reliable, works great for deep learning.

Cons: More complex, needs tuning of extra parameters (beta1, beta2).


In [2]:
import numpy as np
import plotly.graph_objs as go

# ---------------------
# Optimizers
# ---------------------
def batch_gd(X, y, lr=0.01, epochs=30):
    coef = np.ones(X.shape[1])
    intercept = 0
    history = []
    for _ in range(epochs):
        y_hat = X @ coef + intercept
        intercept_der = -2 * np.mean(y - y_hat)
        coef_der = -2 * (X.T @ (y - y_hat)) / X.shape[0]
        intercept -= lr * intercept_der
        coef -= lr * coef_der
        loss = np.mean((y - (X @ coef + intercept)) ** 2)
        history.append((coef[0], coef[1], loss))
    return np.array(history)

def sgd(X, y, lr=0.01, epochs=30):
    coef = np.ones(X.shape[1])
    intercept = 0
    history = []
    for _ in range(epochs):
        for i in range(X.shape[0]):
            idx = np.random.randint(0, X.shape[0])
            y_hat = X[idx] @ coef + intercept
            intercept_der = -2 * (y[idx] - y_hat)
            coef_der = -2 * X[idx] * (y[idx] - y_hat)
            intercept -= lr * intercept_der
            coef -= lr * coef_der
            loss = np.mean((y - (X @ coef + intercept)) ** 2)
            history.append((coef[0], coef[1], loss))
    return np.array(history)

def adam(X, y, lr=0.01, epochs=30, beta1=0.9, beta2=0.999, eps=1e-8):
    coef = np.ones(X.shape[1])
    intercept = 0
    m_coef = np.zeros_like(coef)
    v_coef = np.zeros_like(coef)
    m_int = 0
    v_int = 0
    history = []
    t = 0
    for _ in range(epochs):
        y_hat = X @ coef + intercept
        intercept_der = -2 * np.mean(y - y_hat)
        coef_der = -2 * (X.T @ (y - y_hat)) / X.shape[0]
        t += 1

        # Adam update for intercept
        m_int = beta1 * m_int + (1 - beta1) * intercept_der
        v_int = beta2 * v_int + (1 - beta2) * (intercept_der ** 2)
        m_int_hat = m_int / (1 - beta1 ** t)
        v_int_hat = v_int / (1 - beta2 ** t)
        intercept -= lr * m_int_hat / (np.sqrt(v_int_hat) + eps)

        # Adam update for coef
        m_coef = beta1 * m_coef + (1 - beta1) * coef_der
        v_coef = beta2 * v_coef + (1 - beta2) * (coef_der ** 2)
        m_coef_hat = m_coef / (1 - beta1 ** t)
        v_coef_hat = v_coef / (1 - beta2 ** t)
        coef -= lr * m_coef_hat / (np.sqrt(v_coef_hat) + eps)

        loss = np.mean((y - (X @ coef + intercept)) ** 2)
        history.append((coef[0], coef[1], loss))
    return np.array(history)

# ---------------------
# Generate data
# ---------------------
np.random.seed(42)
X = np.random.randn(200, 2)
true_coef = np.array([3.0, -2.0])
y = X @ true_coef + 5 + np.random.randn(200) * 0.5

# ---------------------
# Run all optimizers
# ---------------------
path_bgd = batch_gd(X, y, lr=0.05, epochs=50)
path_sgd = sgd(X, y, lr=0.01, epochs=5)
path_adam = adam(X, y, lr=0.05, epochs=50)

# ---------------------
# Build loss surface
# ---------------------
w0 = np.linspace(-1, 6, 80)
w1 = np.linspace(-5, 3, 80)
W0, W1 = np.meshgrid(w0, w1)
Loss = np.zeros_like(W0)
for i in range(W0.shape[0]):
    for j in range(W0.shape[1]):
        coef = np.array([W0[i, j], W1[i, j]])
        y_hat = X @ coef + 5
        Loss[i, j] = np.mean((y - y_hat) ** 2)

# ---------------------
# Plotly figure: Optimizer Race
# ---------------------
surface = go.Surface(x=W0, y=W1, z=Loss, colorscale='Viridis', opacity=0.6)

traj_bgd = go.Scatter3d(x=path_bgd[:,0], y=path_bgd[:,1], z=path_bgd[:,2],
                        mode='lines+markers', name='Batch GD', line=dict(color='blue', width=4), marker=dict(size=5))
traj_sgd = go.Scatter3d(x=path_sgd[:,0], y=path_sgd[:,1], z=path_sgd[:,2],
                        mode='lines+markers', name='SGD', line=dict(color='red', width=4), marker=dict(size=5))
traj_adam = go.Scatter3d(x=path_adam[:,0], y=path_adam[:,1], z=path_adam[:,2],
                         mode='lines+markers', name='Adam', line=dict(color='green', width=4), marker=dict(size=5))

fig = go.Figure(data=[surface, traj_bgd, traj_sgd, traj_adam])
fig.update_layout(scene=dict(xaxis_title='Coefficient 0',
                             yaxis_title='Coefficient 1',
                             zaxis_title='Loss (MSE)'),
                  title="Optimizer Race on Loss Surface")

fig.show()
